In [286]:
import pdfplumber
import pandas

In [373]:
pagelines = {
    # Acrisure, non-Steelers
    1: [60, 90, 150, 260, 370, 405, 440, 750, 1075, 1135, 1160],
    # Acrisure, Steelers
    2: [68, 95, 180, 310, 460, 520, 550, 820, 1080, 1135, 1150],

    # PPG, Penguins
    3: [70, 95, 180, 310, 420, 465, 510, 760, 1000, 1090, 1130],
    4: [70, 95, 180, 310, 420, 465, 510, 760, 1000, 1090, 1130],
    # PPG, non-Penguins
    5: [120, 150, 210, 390, 545, 595, 630, 840, 1010, 1070, 1095],
    6: [120, 150, 210, 390, 545, 595, 630, 840, 1010, 1070, 1095],

    # PNC, Pirates
    7: [75, 105, 180, 350, 480, 550, 610, 780, 1000, 1100, 1135],
    8: [75, 105, 180, 350, 490, 550, 610, 820, 1020, 1100, 1135],
    9: [75, 105, 180, 350, 490, 550, 610, 820, 1020, 1100, 1135]
}

In [321]:
pdf = pdfplumber.open("tickets.pdf")

In [374]:
# pdf.pages[6].within_bbox((-14.4, 82, 1209.6, 740)).to_image().debug_tablefinder({
#     "horizontal_strategy": "text",
#     "vertical_strategy": "explicit",
#     "explicit_vertical_lines": [75, 105, 180, 350, 480, 550, 610, 780, 1000, 1100, 1135]
# })

In [375]:
tables = []

In [376]:
for page in pdf.pages:
    crop = page.within_bbox((-14.4, 82, 1209.6, 740))
    table = crop.extract_table({
        "horizontal_strategy": "text",
        "vertical_strategy": "explicit",
        "explicit_vertical_lines": pagelines[page.page_number]
    })

    tables.append(table)

In [387]:
tables = [i for t in tables for i in t]

In [424]:
df = pandas.DataFrame(tables, columns=[
    'date', 'event',
    'suiteRequestor', 'suiteReceiver', 'suiteReason', 'suiteTicketsCount',
    'standsRequestor', 'standsReceiver', 'standsReason', 'standsTicketsCount',
])

suitedf = df[['date', 'event', 'suiteRequestor', 'suiteReceiver', 'suiteReason', 'suiteTicketsCount']]
suitedf.columns = ['date', 'event', 'requestor', 'receiver', 'reason', 'ticketsCount']
suitedf['seatType'] = 'suite'

standsdf = df[['date', 'event', 'standsRequestor', 'standsReceiver', 'standsReason', 'standsTicketsCount']]
standsdf.columns = ['date', 'event', 'requestor', 'receiver', 'reason', 'ticketsCount']
standsdf['seatType'] = 'stands'

df = pandas.concat([suitedf, standsdf])
df = df.drop(df[df.date == ''].index)

/var/folders/m6/nmg96kbd65n3tz43pzrdnnw00000gn/T/ipykernel_47186/1317449178.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suitedf['seatType'] = 'suite'
/var/folders/m6/nmg96kbd65n3tz43pzrdnnw00000gn/T/ipykernel_47186/1317449178.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  standsdf['seatType'] = 'stands'


In [ ]:
# suiteReason: ['', 'Request', 'Donation', 'Economic Impact', 'Request / Donation']
# standsReason: ['', 'Request / Donation', 'Donation', 'Donation / Request',
       # 'Request', 'Request/Donation', 'Donation / Request /Donation',
       # 'Request / Donation / Request /Donation',
       # 'Request / Donation / Donation', 'Request / Request / Donation']

In [437]:
df[(df.reason == '') & (~df.receiver.str.contains('Unused'))].requestor.unique()

array(['vid fans not allowed)', '', 'No Fans - Covid', 'Tickets Returned',
       'Not Available', 'ovid fans not allowed)',
       'Veterans Leadership Program', 'Steve Steingart (8)(Board Member)'],
      dtype=object)

In [441]:
df[~df.receiver.isin([
    'No Fans - Covid', 'Not Available', 'Tickets Returned', 'Unused'
])].reason.value_counts()

reason
Donation                                  477
Request                                   120
Request / Donation                         77
Economic Impact                            20
                                            9
Request/Donation                            6
Donation / Request /Donation                4
Donation / Request                          2
Request / Donation / Request /Donation      1
Request / Donation / Donation               1
Request / Request / Donation                1
Name: count, dtype: int64